# Part C: Self-Taught Learning

## Installing packages

In [1]:
#installing package h2o for performing autoencoder
install.packages("h2o", type="source", repos="http://h2o-release.s3.amazonaws.com/h2o/rel-wright/9/R")  

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
#installing package BBmisc for normalization
install.packages("BBmisc")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Importing Libraries

In [3]:
library(h2o)


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc



In [4]:
library(BBmisc)


Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [5]:
library(tidyr)

In [6]:
#if there exists proxy already, getting its environment. otherwise setting the new environment.
proxy.old <- Sys.getenv('http_proxy'); Sys.setenv('http_proxy'='');
localH2O <-  h2o.init(nthreads = -1, port = 54321, max_mem_size = '6G', startH2O = TRUE) #starting h2o cluster


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/Rtmpxkj742/h2o_nbuser_started_from_r.out
    /tmp/Rtmpxkj742/h2o_nbuser_started_from_r.err


Starting H2O JVM and connecting: ....... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         12 seconds 312 milliseconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.20.0.9 
    H2O cluster version age:    1 year, 8 months and 10 days !!! 
    H2O cluster name:           H2O_started_from_R_nbuser_ddi832 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   5.33 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost,

Warning message in h2o.clusterInfo():
“
Your H2O cluster version is too old (1 year, 8 months and 10 days)!
Please download and install the latest version from http://h2o.ai/download/”

## Loading the data sets

In [7]:
#read the labeled and unlabed data sets
label_data <- read.csv("Task2C_labeled.csv")
unlabel_data <- read.csv("Task2C_unlabeled.csv")

In [8]:
#printing first few lines of labeled data
head(label_data)

label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#printing first few lines of unlabeled data
head(unlabel_data)

label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Inorder to see the expected behavior in the graph remove NA entries and normalizate the data sets

In [10]:
#removing NA's using drop_na
label_data <- label_data%>% drop_na
unlabel_data <- unlabel_data%>%drop_na

In [11]:
#labelling label and unlabel data
dim(label_data)
dim(unlabel_data)

[1]  50 785

[1] 100 785

In [12]:
## factorize the labels of the labelled and unlabelled data so as to make it easier to train the models for reconstruction and classification using Neural Networks.
label_data$label <- as.factor(label_data$label)
unlabel_data$label <- as.factor(unlabel_data$label)

In [13]:
#normalizing the data using normalize function
label_data <- normalize(label_data, method = "standardize", range = c(0, 1), margin = 2L, on.constant = "quiet")
unlabel_data <- normalize(unlabel_data, method = "standardize", range = c(0, 1), margin = 2L, on.constant = "quiet")

In [14]:
head(label_data) #printing first few lines of labeled data

label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
head(unlabel_data) #printing first few lines of unlabeled data

label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#write the formatted labeled and unlabeled data into a csv file
write.csv(label_data, "Task2C_labeled_formatted.csv", row.names = FALSE)
write.csv(unlabel_data , "Task2C_unlabeled_formatted.csv", row.names = FALSE)

In [17]:
#defining error function to calculate the number of missclassification rates
error.rate <- function(Y1, T1){
  if (nrow(Y1)!=nrow(T1)){
    stop('error.rate: size of true lables and predicted labels mismatch')
  }
  return (sum(T1!=Y1)/nrow(T1))
}

### Question 3a. Load files required for training the autoencoder

In [18]:
# reading the formatted csv files using h2o as we need to perform autoencoding functionality
labeled.frame <- h2o.importFile(path = 'Task2C_labeled_formatted.csv' ,sep=',') 
unlabeled.frame <- h2o.importFile(path = 'Task2C_unlabeled_formatted.csv' ,sep=',') 
test.frame <- h2o.importFile(path = 'Task2C_test.csv' ,sep=',') 

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [19]:
labeled.frame[,1] <- as.factor(labeled.frame$label)
unlabeled.frame[,1] <- NA
train.frame <- h2o.rbind(labeled.frame[,-1], unlabeled.frame[,-1])
test.frame[,1] <- as.factor(test.frame$label)

### Question3b.  Train an autoencoder with only one hidden layer and change the number of its neurons to: 20, 40, 60, 80, …, 400 (i.e. from 20 to 400 with a step size of 20).

In [21]:
options(warn = -1)
for (k in seq(20, 400, 20)){
    
    NN.model <- h2o.deeplearning(    
        
        x = 2:ncol(train.frame),
        training_frame = train.frame,     
        hidden = c(k), 
        epochs = 50, 
        activation = 'Tanh', 
        autoencoder = TRUE, 
        l2 = 0.1
    )
}
NN.model

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |=====================================

Model Details:

H2OAutoEncoderModel: deeplearning
Model ID:  DeepLearning_model_R_1591928694903_28 
Status of Neuron Layers: auto-encoder, gaussian distribution, Quadratic loss, 404,905 weights/biases, 4.7 MB, 7,581 training samples, mini-batch size 1
  layer units  type dropout       l1       l2 mean_rate rate_rms momentum
1     1   505 Input  0.00 %       NA       NA        NA       NA       NA
2     2   400  Tanh  0.00 % 0.000000 0.100000  0.010329 0.010107 0.000000
3     3   505  Tanh      NA 0.000000 0.100000  0.006330 0.001938 0.000000
  mean_weight weight_rms mean_bias bias_rms
1          NA         NA        NA       NA
2    0.000087   0.025084 -0.000763 0.010878
3    0.000116   0.026932  0.004188 0.010430


H2OAutoEncoderMetrics: deeplearning
** Reported on training data. **

Training Set Metrics: 

MSE: (Extract with `h2o.mse`) 0.008658985
RMSE: (Extract with `h2o.rmse`) 0.09305367




### Question 3c. For each model in 3b, calculate and record the reconstruction error which is simply the average (over all data points while the model is fixed) of Euclidian distances between the input and output of the autoencoder (you can simply use “h2o.anomaly()” function).

In [25]:
set.seed(1234)
reconstruction_error <- data.frame('K' = seq(20, 400, 20) , 'Test' = rep(0,20))

# build a neural network classifier based on the train dataframe i.e. labeled and unlabeled, where number of neurons in the hidden layer changes from
# 20 to 400, with the step size 20. 

for (k in seq(20, 400, 20)){
    
    NN.model <- h2o.deeplearning(    
        
        x = 2:ncol(train.frame),
        training_frame = train.frame, #specify the train dataframe
        hidden = c(k), #no.of units in the dataframe
        epochs = 50, #number of iterations
        activation = 'Tanh', #activation function to be used
        autoencoder = TRUE, #autoencoder is needed or not
        l2 = 0.1
    )
    
    #record the reconstruction error which is an average over all the data points while the model is fixed i.e. the euclidean distance between the input 
    #output of the autoencoder
    reconstruction_error$Test[reconstruction_error$K == k] <- mean(h2o.anomaly(NN.model, train.frame))
}

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |=====================================

In [26]:
#printing the reconstruction error dataframe
reconstruction_error

K,Test
<dbl>,<dbl>
20,0.028372109
40,0.015483593
60,0.011731838
80,0.010886823
100,0.010956119
120,0.011177460
140,0.011797544
160,0.012070445
180,0.011591732


### Plot the graph between x-axis is the number of units in the middle layer and the y-axis is the reconstruction error.

In [37]:
library(ggplot2)
options(repr.plot.width = 10, repr.plot.height = 8)
ggplot(data = reconstruction_error, aes(x = K, y = Test)) + geom_line() +
ggtitle("Reconstruction error-rate with respect to no.of hidden layers using autoencoder")

### Question 3d. Build the 3-layer NN from Activity 5.1 or “h2o.deeplearning” function (make sure you set “ autoencoder = FALSE”) to build a classification model using all the original attributes from the training set and change the number of its neurons to: 20, 40, 60, 80, .., 400 like Step II. For each model, calculate and record the test error.

In [25]:
options(warn = -1)
set.seed(1234)
classification_error <- data.frame('K' = seq(20, 400, 20) , 'Test' = rep(0,20))

#here we take only labeled.frame as our training frame as we are setting our autoencoder = FALSE
for (k in seq(20, 400, 20)){
    
    NN.model <- h2o.deeplearning(    
        
        x = 2:ncol(labeled.frame), 
        y = 1,
        training_frame = labeled.frame, 
        hidden = c(k), #number of neurons in hidden layer changes with respect to k
        epochs = 50,  #number of iterations
        activation = 'Tanh',  #activation function used
        autoencoder = FALSE, #autoencoder is set to FALSE
        l2 = 0.1
        
   )
    
    #using predict function calculating the test error for test data
    test.predict <- h2o.predict(NN.model, test.frame)$predict
    classification_error$Test[classification_error$K == k] <- error.rate(test.frame$label, test.predict)
}

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |=====================================

In [35]:
#printing classification error dataframe
classification_error

K,Test
<dbl>,<dbl>
20,0.687
40,0.401
60,0.678
80,0.649
100,0.651
120,0.584
140,0.607
160,0.680
180,0.560


### Plot the error rates for 3-layer neural network where x = number of extra features and y = classification error

In [36]:
options(repr.plot.width = 10, repr.plot.height = 6)
ggplot(data = classification_error, aes(x = K, y = Test)) + geom_line() + 
ggtitle("Classification error-rate with respect to no.of hidden layers for three layer neural network ")

### Question 3e. Build augmented self-taught networks using the models learnt in Step 3b. For each model:
### A. Add the output of the middle layer as extra features to the original feature set.
### B. Train a 3-layer NN (similar to Step IV) using all features (original + extra). Then calculate and record the test error.

In [ ]:
options(warn = -1)
set.seed(1234)
#building an empty dataframe
classification_error_threelayer_network <- data.frame('K' = seq(20, 400, 20) , 'Test' = rep(0,20))

#We need to add the output of autoencoder i.e middle layer as extra features to the original feature set. For this purpose, we first set autoencoder to TRUE
#and the output of this function is passed is combined with labeled data frame and is passed as an input to three layer neural netwoek model.
#In the first case as we are using the autoencoder, we will set its parameter value to TRUE, as the functionality of autoencoder is using both the labeled
#and unlabeled data and predicting the input again.
for (k in seq(20, 400, 20)){
    
    NN.model.output.autoencoder <- h2o.deeplearning(    
        
        x = 2:ncol(train.frame),
        training_frame = train.frame,     
        hidden = c(k), 
        epochs = 50, 
        activation = 'Tanh', 
        autoencoder = TRUE, 
        l2 = 0.1
    )
    
    #features of autoencoder is retrieved using h2o.deepfeatures and is binded with labeled data 
    labeled_extra_features <- h2o.cbind(labeled.frame, as.numeric(h2o.deepfeatures(NN.model.output.autoencoder, labeled.frame, layer = 1)))
    
    #the obtained extra features are taken here as training frame and three layer neural network model is built again.
    NN.model.threelayer.neural.network <- h2o.deeplearning(    
        
        x = 2:ncol(labeled_extra_features), 
        y = 1,
        training_frame = labeled_extra_features,    
        hidden = c(k), 
        epochs = 50,  
        activation = 'Tanh',  
        autoencoder = FALSE,
        l2 = 0.1
        
   )
    
    #predicting the test label using the h2o.predict function for building three layer neural network model.
    test.predict <- h2o.predict(NN.model.threelayer.neural.network, test.frame)$predict
    #using error.rate function calculating the error rate for estimated labels with ground truth labels of test data
    classification_error_threelayer_network$Test[classification_error_threelayer_network$K == k] <- error.rate(test.frame$label, test.predict)
    
}

In [21]:
#printing classification error dataframe for augumented self-taught networks.
classification_error_threelayer_network

K,Test
<dbl>,<dbl>
20,0.511
40,0.686
60,0.649
80,0.479
100,0.553
120,0.663
140,0.597
160,0.666
180,0.677


### Question 3f. Plot the error rates for augumented self-taught networks where x - axis is the number of extra features and y - axis is the test error rate

In [24]:
library(ggplot2)
options(repr.plot.width = 10, repr.plot.height = 6)
ggplot(data = classification_error_threelayer_network, aes(x = K, y = Test)) + geom_line(linetype="dashed",color="blue", size=0.5) + geom_point() +
ggtitle("Classification error-rate with respect to no.of hidden layers to augumented self-taught networks")